In [ ]:
import onnx
import onnx_caffe2.backend

In [ ]:
model = onnx.load("torchmodel.onnx")

In [ ]:
prepared_backend = onnx_caffe2.backend.prepare(model)

In [ ]:
import onnx
import caffe2.python.onnx.backend

# Prepare the inputs, here we use numpy to generate some random inputs for demo purpose
import numpy as np
img = np.zeros((1, 3, 320, 1152), dtype=np.float32)

# Load the ONNX model
model = onnx.load('torchmodel.onnx')
# Run the ONNX model with Caffe2
outputs = caffe2.python.onnx.backend.run_model(model, [img])

In [ ]:
convert-onnx-to-caffe2 assets/squeezenet.onnx --output predict_net.pb --init-net-output init_net.pb

In [ ]:
# Some standard imports
from caffe2.proto import caffe2_pb2
from caffe2.python import core, net_drawer, net_printer, visualize, workspace, utils

In [ ]:
# Let's run the mobile nets that we generated above so that caffe2 workspace is properly initialized
workspace.RunNetOnce("init_net.pb")
workspace.RunNetOnce(predict_net)

In [1]:
# Some standard imports
from caffe2.proto import caffe2_pb2
from caffe2.python import core, net_drawer, net_printer, visualize, workspace, utils

# run caffe2 inference
with open('old_init_net.pb', 'rb') as f:
    init_net = f.read()
with open('old_predict_net.pb', 'rb') as f:
    predict_net = f.read()

predictor = workspace.Predictor(init_net, predict_net)


In [ ]:
import matplotlib.pyplot as plt

In [2]:
from PIL import Image

def load_img(ids, directory, suffix):
    for iid in ids:
        img = np.array(Image.open(directory + iid + suffix))
        yield img[:320,:1152]
        
# In[257]:
def hwc_to_chw(img):
    return np.transpose(img, axes=[2, 0, 1])

# In[258]:
def normalize(x):
    return np.float32(x/255.)

In [10]:
import numpy as np
import torch

file = "/home/kenny/Desktop/lanes-segmentation/data/images/um/um_000000.png"
x = np.array(Image.open(file))
x = x[:256,:256]
x = hwc_to_chw(x)
x = normalize(x)
x = np.expand_dims(x, axis = 0)
# x = torch.from_numpy(x)

In [11]:
x.shape

(1, 3, 256, 256)

In [12]:
import time

prev_time = time.time()
output = predictor.run([x])
current_time = time.time()
print(current_time-prev_time)


2.7532529830932617


In [ ]:
np.array(output).shape

In [ ]:
out = np.array(output)[0,0,0,:,:]

In [ ]:
msk = out > 0.5

In [ ]:
plt.imshow(msk)